<a href="https://colab.research.google.com/github/tejugl/projects/blob/master/R8_External_Lab__teju.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
import keras
from keras.datasets import cifar10

In [3]:
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [4]:
x_train.shape

(50000, 32, 32, 3)

In [5]:
y_train.shape

(50000, 1)

In [6]:
x_test.shape

(10000, 32, 32, 3)

In [0]:
import numpy as np

In [0]:
x_train_lt5 = x_train[y_train.reshape(y_train.shape[0])<5]
y_train_lt5 = y_train[y_train.reshape(y_train.shape[0])<5]
x_test_lt5 = x_test[y_test.reshape(y_test.shape[0])<5]
y_test_lt5 = y_test[y_test.reshape(y_test.shape[0])<5]

x_train_gt5 = x_train[y_train.reshape(y_train.shape[0]) >= 5]
y_train_gt5 = y_train[y_train.reshape(y_train.shape[0]) >= 5] - 5  
x_test_gt5 = x_test[y_test.reshape(y_test.shape[0]) >= 5]         
y_test_gt5 = y_test[y_test.reshape(y_test.shape[0]) >= 5] - 5

In [9]:
print(x_train_lt5.shape)
print(y_train_lt5.shape)
print(x_test_lt5.shape)
print(y_test_lt5.shape)

(25000, 32, 32, 3)
(25000, 1)
(5000, 32, 32, 3)
(5000, 1)


In [10]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(25000, 32, 32, 3)
(25000, 1)
(5000, 32, 32, 3)
(5000, 1)


In [0]:
x_train_lt5_normalized=x_train_lt5.astype(float)/255

In [0]:
x_test_lt5_normalized=x_test_lt5.astype(float)/255

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_train_lt5_ohe=keras.utils.to_categorical(y_train_lt5)

In [14]:
y_train_lt5_ohe.shape

(25000, 5)

In [0]:
y_test_lt5_ohe=keras.utils.to_categorical(y_test_lt5)

In [16]:
y_test_lt5_ohe.shape

(5000, 5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
from keras.models import Sequential
from keras.layers import Dense,Convolution2D,MaxPooling2D,Dropout,Activation,Flatten

In [18]:
model=Sequential()

model.add(Convolution2D(32,(3,3),input_shape=(32,32,3)))
model.add(Activation('relu'))

model.add(Convolution2D(64,(3,3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Convolution2D(128,(3,3)))
model.add(Activation('relu'))

model.add(Convolution2D(256,(3,3)))
model.add(Activation('relu'))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))
          
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

W0720 16:23:59.779442 139882693642112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0720 16:23:59.829490 139882693642112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0720 16:23:59.840519 139882693642112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0720 16:23:59.904780 139882693642112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0720 16:23:59.908005 139882693642112 deprecation_wrapp

In [0]:
model.fit(x_train_lt5_normalized,y_train_lt5_ohe,validation_data=(x_test_lt5_normalized,y_test_lt5_ohe),epochs=5)

W0720 16:24:00.248180 139882693642112 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 5000 samples
Epoch 1/5
18912/25000 [=====================>........] - ETA: 1:06 - loss: 1.0879 - acc: 0.5430

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
model.summary()

In [0]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False
        print(layers.name + 'is not trainable\n')
    if('dense' in layers.name):
        print(layers.name + ' is trainable\n')

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
x_train_gt5_normalized=x_train_gt5.astype(float)/255

In [0]:
x_test_gt5_normalized=x_test_gt5.astype(float)/255

In [0]:
y_train_gt5=keras.utils.to_categorical(y_train_gt5)

In [0]:
y_test_gt5=keras.utils.to_categorical(y_test_gt5)

In [0]:
model.fit(x_train_gt5_normalized,y_train_gt5,validation_data=(x_test_gt5_normalized,y_test_gt5),nb_epoch=5)

## Sentiment analysis <br> 

The objective of the second problem is to perform Sentiment analysis from the tweets data collected from the users targeted at various mobile devices.
Based on the tweet posted by a user (text), we will classify if the sentiment of the user targeted at a particular mobile device is positive or not.

### 6. Read the dataset (tweets.csv) and drop the NA's while reading the dataset

In [0]:
import pandas as pd
data = pd.read_csv('./tweets.csv', encoding = "ISO-8859-1").dropna()

In [0]:
data.shape

In [0]:
data.head()

### Consider only rows having Positive emotion and Negative emotion and remove other rows from the dataframe.

In [0]:
data = data[(data['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Positive emotion') | (data['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Negative emotion')]

In [0]:
data.shape

### 7. Represent text as numerical data using `CountVectorizer` and get the document term frequency matrix

#### Use `vect` as the variable name for initialising CountVectorizer.

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
vect=CountVectorizer()

In [0]:
vect.fit(data['tweet_text'])

### 8. Find number of different words in vocabulary

In [0]:
len(vect.get_feature_names())

#### Tip: To see all available functions for an Object use dir

In [0]:
dir(vect)

### Find out how many Positive and Negative emotions are there.

Hint: Use value_counts on that column

In [0]:
pd.value_counts(data['is_there_an_emotion_directed_at_a_brand_or_product'])

###  Change the labels for Positive and Negative emotions as 1 and 0 respectively and store in a different column in the same dataframe named 'label'

Hint: use map on that column and give labels

In [0]:
data['label'] = data.is_there_an_emotion_directed_at_a_brand_or_product.map({'Positive emotion':1, 'Negative emotion':0})

### 9. Define the feature set (independent variable or X) to be `text` column and `labels` as target (or dependent variable)  and divide into train and test datasets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train,X_test,Y_train,Y_test=train_test_split(data['tweet_text'],data['label'],test_size=0.2,random_state=1)

In [0]:
vect.fit(X_train)

In [0]:
len(vect.get_feature_names())

In [0]:
X=vect.transform(X_train)

In [0]:
Y_train.shape

In [0]:
test=vect.transform(X_test)

## 10. **Predicting the sentiment:**


### Use Naive Bayes and Logistic Regression and their accuracy scores for predicting the sentiment of the given text

In [0]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()

In [0]:
nb.fit(X,Y_train)

In [0]:
from sklearn.metrics import accuracy_score

In [0]:
y_pred_nb=nb.predict(test)

In [0]:
accuracy_score(Y_test,y_pred_nb)

In [0]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()

In [0]:
lr.fit(X,Y_train)

In [0]:
y_pred_lr=lr.predict(test)

In [0]:
accuracy_score(Y_test,y_pred_lr)

## 11. Create a function called `tokenize_predict` which can take count vectorizer object as input and prints the accuracy for x (text) and y (labels)

In [0]:
def tokenize_test(vect):
    x_train_dtm = vect.fit_transform(X_train)
    print('Features: ', x_train_dtm.shape[1])
    x_test_dtm = vect.transform(X_test)
    nb = MultinomialNB()
    nb.fit(x_train_dtm, Y_train)
    y_pred_class = nb.predict(x_test_dtm)
    print('Accuracy: ', accuracy_score(Y_test, y_pred_class))

### Create a count vectorizer function which includes n_grams = 1,2  and pass it to tokenize_predict function to print the accuracy score

In [0]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test(vect)

### 12. Create a count vectorizer function with stopwords = 'english'  and pass it to tokenize_predict function to print the accuracy score

In [0]:
vect = CountVectorizer(stop_words = 'english')
tokenize_test(vect)

### 13. Create a count vectorizer function with stopwords = 'english' and max_features =300  and pass it to tokenize_predict function to print the accuracy score

In [0]:
vect = CountVectorizer(stop_words = 'english',max_features=300)
tokenize_test(vect)

### 14. Create a count vectorizer function with n_grams = 1,2  and max_features = 15000  and pass it to tokenize_predict function to print the accuracy score

In [0]:
vect = CountVectorizer(ngram_range=(1, 2),max_features=15000)
tokenize_test(vect)

### 15. Create a count vectorizer function with n_grams = 1,2  and include terms that appear at least 2 times (min_df = 2)  and pass it to tokenize_predict function to print the accuracy score

In [0]:
vect = CountVectorizer(ngram_range=(1, 2),min_df=2)
tokenize_test(vect)